# Goal

This notebook walks you through the theory and assumptions behind bayesian linear regression.
Afterwards, it shows how this inference can be done using `skpro`'s `BayesianConjugateLinearRegressor` class.

# Introduction

## Bayesian linear regression

**Bayesian linear regression** is a probabilistic approach to linear regression where prior beliefs about the model parameters are combined with observed data to compute posterior distributions. Unlike traditional linear regression, which provides point estimates for parameters, Bayesian linear regression offers distributions, capturing uncertainty in both parameters and predictions.

In this notebook, we will specifically focus on **Bayesian Linear Regression**, assuming a **multivariate normal distribution** as the prior for the regression coefficients, leveraging its properties as a **conjugate prior** to simplify inference.

## Conjugate Prior

The use of **conjugate priors** simplifies the Bayesian inference. Conjugacy ensures that the posterior distribution belongs to the same family as the prior, making it analytically tractable. 


In our case, as we are using a Gaussian prior and we are dealing with a Gaussian likelihood, the posterior remains Gaussian, allowing for straightforward updates to the mean and covariance of the distribution. This efficiency avoids the need for computationally intensive methods like Monte Carlo Markov Chain sampling.

# Theory


In the sections that follow, we will delve into the key distributions that play a crucial role in Bayesian inference. These include:

1. **The Likelihood Function**: This captures the relationship between the observed data and the model parameters

2. **The Prior Distribution**: The multivariate normal prior represents our initial beliefs about the regression coefficients before observing any data. 

3. **The Posterior Distribution**: By combining the prior and likelihood using Bayes' theorem, we compute the posterior, which reflects our updated beliefs about the coefficients after accounting for the observed data.

4. **The Predictive Distribution**: Finally, we use the posterior to make predictions on new data, incorporating the uncertainty in the coefficients to provide confidence intervals around the predictions.

## Likelihood

### Single target data point $t$

In Bayesian Linear Regression, we assume that the target variable $t$ is generated by a deterministic function $y(\mathbf{x}, \mathbf{w})$, which represents the model's prediction based on the input features $\mathbf{x}$ and the regression coefficients $\mathbf{w}$. This deterministic prediction is subject to additive Gaussian noise, $ epsilon$, which accounts for uncertainty not captured by the deterministic function. Mathematically, this relationship is expressed as:

$$
t = y(\mathbf{x}, \mathbf{w}) + \epsilon,
$$

where $\epsilon \sim \mathcal{N}(0, \beta^{-1})$ follows a Gaussian distribution with zero mean and variance $ \beta^{-1}$. 


We could also reframe this by assuming that $t$ is given probabilistically by a Gaussian distribution:

$$
p(t | \mathbf{x}, \mathbf{w}, \beta) = \mathcal{N}(t | y(\mathbf{x}, \mathbf{w}), \beta^{-1})
$$


### A set of data points $\mathbf{t}$


Now let's consider a data set of inputs $ \mathbf{X} = \{ \mathbf{x}_1, \ldots, \mathbf{x}_N \} $ with corresponding target values $ t_1, \ldots, t_N $. We group the target variables $\{ t_n \}$ into a column vector that we denote by $\mathbf{t}$. 

We assume that these data points are drawn independently from the above distribution. With this assumption, we proceed to construct the following expression for the likelihood function of the whole dataset:

$$
\begin{aligned}
p(\mathbf{t} | \mathbf{X}, \mathbf{w}, \beta) &= \prod_{n=1}^N \mathcal{N}(t_n | \mathbf{w}^T \mathbf{x}_n, \beta^{-1}) \\
&\propto \exp \left( -\frac{\beta}{2} \sum_{n=1}^N (t_n - \mathbf{w}^T \mathbf{x}_n)^2 \right)
\\
&\propto \exp \left( -\frac{\beta}{2} \|\mathbf{t} - \mathbf{X} \mathbf{w}\|^2 \right)
\end{aligned} 
$$


Note: since the data matrix $\mathbf{X} $ always appears in the set of conditioning variables, from this point onwards, we will drop the explicit "$\mathbf{X}$" from expressions such as $ p(\mathbf{t} | \mathbf{X}, \mathbf{w}, \beta) $ to keep the notation uncluttered.


## Prior

We will first introduce a **prior** over the model parameters $\mathbf{w}$, which represents our initial belief about their values before observing any data.

In our framework, we treat the noise precision parameter $\beta$ as a known constant. This simplifies the model while still allowing us to capture uncertainty in the regression coefficients $\mathbf{w}$.

As we saw above, the likelihood function $p(\mathbf{t} | \mathbf{w}, \beta)$ is Gaussian. To achieve conjugacy and facilitate computation, we choose a multivariate Gaussian distribution as the prior for $\mathbf{w}$, given by:


$$
\begin{aligned}
p(\mathbf{w}) &= \mathcal{N}(\mathbf{w} | \mathbf{m}_0, \mathbf{S}_0) \\
&\propto \exp \left( -\frac{1}{2} (\mathbf{w} - \mathbf{m}_0)^T \mathbf{S}_0^{-1} (\mathbf{w} - \mathbf{m}_0) \right)
\end{aligned}
$$


where:
- $\mathbf{m}_0$ is the prior mean vector of the regression coefficients; Its shape is $(D, 1)$, where $D$ is the number of features or coefficients.
- $\mathbf{S}_0$ is the prior covariance matrix; its shape is $(D, D)$.


**Simplification**

To further simplify treatment, we shall use a particular form of a multivariate Gaussian prior.

The prior we'll use is a **zero-mean isotropic Gaussian**, where the prior mean $\mathbf{m}_0$ is zero and the prior covariance $\mathbf{S}_0$ is isotropic (i.e. the distribution has the same variance in all directions). 

This distribution is governed by a single scalar precision parameter $\alpha$ so that:

$$
p(\mathbf{w} | \alpha) = \mathcal{N}(\mathbf{w} | 0, \alpha^{-1} \mathbf{I})
$$



## Posterior

**General form**

According to the Bayes formula, the **posterior** is proportional to the product of the likelihood and the prior:
<br>

$$
p(\mathbf{w} | \mathbf{t}) \propto \exp \left( -\frac{\beta}{2} \|\mathbf{t} - \mathbf{X} \mathbf{w}\|^2 \right) \exp \left( -\frac{1}{2} (\mathbf{w} - \mathbf{m}_0)^T \mathbf{S}_0^{-1} (\mathbf{w} - \mathbf{m}_0) \right)
$$

After expanding the terms in the exponents and completing the square, we obtain a posterior that's also a multivariate Gaussian:

$$
p(\mathbf{w} | \mathbf{t}) = \mathcal{N}(\mathbf{w} | \mathbf{m}_N, \mathbf{S}_N)
$$

where:
- $\mathbf{S}_N$ is the posterior covariance. Its inverse (posterior precision) is given by:
  $$
  \mathbf{S}_N^{-1} = \mathbf{S}_0^{-1} + \beta \mathbf{X}^T \mathbf{X}
  $$
- $\mathbf{m}_N$ is the posterior mean, given by:
  $$
  \mathbf{m}_N = \mathbf{S}_N \left( \mathbf{S}_0^{-1} \mathbf{m}_0 + \beta \mathbf{X}^T \mathbf{t} \right)
  $$


**Simplification**

As mentioned above, we'll use a special case with the following parameters for our Gaussian prior:
- prior precision $\mathbf{S}_0^{-1} = \alpha \mathbf{I}$ 
- prior mean $\mathbf{m}_0 = \mathbf{0}$ 

For this special case, the posterior precision simplifies to:
$$
\mathbf{S}_N^{-1} = \alpha \mathbf{I} + \beta \mathbf{X}^T \mathbf{X}
$$

The posterior mean simplifies to:
$$
\mathbf{m}_N = \beta \mathbf{S}_N \mathbf{X}^T \mathbf{t}
$$

**Intuition**

1. **Posterior Precision $\mathbf{S}_N^{-1}$**:

    - The posterior precision $\mathbf{S}_N^{-1}$ is the sum of the prior-derived precision and data-derived precision.
    - The prior precision $\mathbf{S}_0^{-1} = \alpha \mathbf{I}$ reflects initial uncertainty in the weights $\mathbf{w}$.
    - The data precision $\beta \mathbf{X}^T \mathbf{X}$ reflects the amount of information provided by the observed data $\mathbf{X}$, adjusted by the noise precision $\beta$.



2. **Posterior Mean ($\mathbf{m}_N$)**:
   - Since we assume the prior mean $\mathbf{m}_0$ to be 0, the posterior mean comes exclusively from the data



## Posterior Predictive

Our ultimate goal is to get the posterior predictive distribution of a new target $t$ given a new input $\mathbf{x}$:

$$
p(t | \mathbf{x}, \mathbf{X}, \mathbf{t}) = \mathcal{N}(t | m(\mathbf{x}), s^2(\mathbf{x}))
$$

As the notation suggests, this distribution depends on the training data ($\mathbf{X}$ and $\mathbf{t}$) used to fit the model.

This posterior predictive distribution is a univariate Gaussian with mean $m(\mathbf{x})$ and variance $s^2(\mathbf{x})$, both of which depend on the given input $\mathbf{x}$.


### **Predictive Mean**

The predictive mean $m(\mathbf{x})$ is given by:
$$
\begin{aligned}
m(\mathbf{x}) &= \mathbf{x}^T \beta S_N \mathbf{X}^T \mathbf{t} \\
&= \mathbf{x}^T \mathbf{m}_N
\end{aligned}
$$

We note that this predictive mean is very simple: it is simply a projection of incoming data point $\mathbf{x}$ onto the posterior mean $\mathbf{m}_N$.


### **Predictive Variance**

The predictive variance $s^2(\mathbf{x})$ is given by:
$$
s^2(\mathbf{x}) = \beta^{-1} + \mathbf{x}^T S_N \mathbf{x}
$$


Predictive variance quantifies model confidence, increasing in regions far from the training data or in uncertain directions. From the formula, we note that this uncertainty depends on the **position** and **direction** of the incoming data point $\mathbf{x}$:

- If it's close to fitted training data: $\mathbf{x}^T S_N \mathbf{x}$ is small near training data, where the model is confident.
- On the other hand, if it is far from fitted training data: $\mathbf{x}^T S_N \mathbf{x}$ grows as $\mathbf{x}$ moves away, reflecting increased uncertainty.
- Lastly, larger $\|\mathbf{x}\|$ increases $\mathbf{x}^T S_N \mathbf{x}$, leading to higher variance.


# Application

The above framework is implemented by the `BayesianConjugateLinearRegressor` class from `skpro`. In this section, we'll take a look at its usage.

In [1]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from skpro.regression.bayesian.bayesian_conjugate import (
    BayesianConjugateLinearRegressor,
)

## Data

We will first load our dataset using the `load_diabetes` function from `sklearn`. 
We will then split this dataset into 

In [2]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X = X.iloc[:10]
y = y.iloc[:10]
y = pd.DataFrame(y)

X_train_update, X_test, y_train_update, _ = train_test_split(X, y, random_state=42)
X_train, X_update, y_train, y_update = train_test_split(
    X_train_update, y_train_update, random_state=42
)

In [3]:
X_train

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
9,-0.070900,-0.044642,0.039062,-0.033213,-0.012577,-0.034508,-0.024993,-0.002592,0.067737,-0.013504
6,-0.045472,0.050680,-0.047163,-0.015999,-0.040096,-0.024800,0.000779,-0.039493,-0.062917,-0.038357


## Instantiation and Fitting

We first instantiate a `BayesianConjugateLinearRegressor` model object with `alpha` = 0.5 and `beta` = 3

In [4]:
model = BayesianConjugateLinearRegressor(alpha=0.5, beta=3)

After performing `.fit`, we will have access to a posterior distribution and its parameters (`mu` and `cov`)

In [5]:
model.fit(X_train, y_train)

BayesianConjugateLinearRegressor(alpha=0.5, beta=3)

We see that the prior `mu` is zero, as explained above in our assumption.

In [6]:
model._prior_mu

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

Meanwhile, the prior covariance is the identity matrix times 1/`alpha`:

In [7]:
model._prior_cov

array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

As expected, the posterior has the same shape as the prior:

In [8]:
model._posterior_mu.shape

(10, 1)

In [9]:
model._posterior_cov.shape

(10, 10)

## Prediction

We can then use our fitted model to perform prediction.
The resulting prediction is an instance of `skpro`'s `Normal` distribution with the same size as our incoming data `X_test`.

In [10]:
X_test

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
8,0.041708,0.050680,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014960,0.011349
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
5,-0.092695,-0.044642,-0.040696,-0.019442,-0.068991,-0.079288,0.041277,-0.076395,-0.041176,-0.096346


In [11]:
y_test_pred_proba = model.predict_proba(X_test)

In [12]:
y_test_pred_proba

Normal(columns=Index(['target'], dtype='object'),
       index=Index([8, 1, 5], dtype='int64'),
       mu=[[-4.390691198701689], [6.064281031619252], [35.57270620266341]],
       sigma=[[0.5949424812347218], [0.6186268611569907], [0.6419937217502341]])

In [13]:
y_test_pred_proba.shape

(3, 1)

In [14]:
y_test_pred = model.predict(X_test)

In [15]:
y_test_pred

,target
8,-4.390691
1,6.064281
5,35.572706
